In [1]:
import requests
from fake_useragent import UserAgent
import json
from tqdm import tqdm
from time import sleep
# fake useragent
ua = UserAgent()

user_agent = ua.random


# add headers 模拟浏览器正常访问请求
headers = {
    "User-Agent": user_agent,
    "Cookie": r"ttwid=1%7CmyHgF6b1xI5ibG3ITgfcq5ZVvIBbT6DE4VNW2h-JS_I%7C1721306154%7Ccead4d9da5b5fe6ae95fd319f1528c3b7199bb03307759ed95e0e3ce3812be28; tt_webid=7392953585342842430; tt_web_version=new; s_v_web_id=verify_lyr98o17_Df8Qunzk_DC92_4Rhh_B5MA_ptTG9CSea0WY; _gid=GA1.2.743883221.1721306153; city_name=%E9%87%8D%E5%BA%86; is_dev=false; is_boe=false; Hm_lvt_3e79ab9e4da287b5752d8048743b95e6=1721309659; HMACCOUNT=8160C1392AF61C82; Hm_lpvt_3e79ab9e4da287b5752d8048743b95e6=1721309665; msToken=-A6plihpMHPZ8KdBbgtAgwfzXN7BC68xK4EoeslAj10vvTlg-P6bDp7jDLu92wErp-fezviZiY9-45b_zD6QaTvYmWoMiFRZHXhPgXpl; _gat_gtag_UA_138671306_1=1; _ga=GA1.1.427450835.1721306153; _ga_YB3EWSDTGF=GS1.1.1721356560.2.1.1721356581.39.0.0",
}
# add data

data = [{
        "sh_city_name": "全国"
         , "page": int("{}".format(i))
         , "limit": 20
         } for i in range(1, 500 + 1, 1)]

# net
href = (
    r"https://www.dongchedi.com/motor/pc/sh/sh_sku_list?aid=1839&app_name=auto_web_pc"
)



# response
for _ in tqdm(range(len(data))):

    res = requests.post(href, headers=headers, data=data[_])

    # return status code
    res.raise_for_status

    # judge content encoding
    res.encoding = res.apparent_encoding
    # print(res.json())
    sleep(1)
    ## write middle
    with open ('./json/cars{}.json'.format(_+1), 'w', encoding = "utf-8") as f:
        json.dump(res.json(), f)


100%|██████████| 500/500 [16:54<00:00,  2.03s/it]


In [2]:
# replace
import re

wordMap = {
    r'\ue3eb':'7',
    r'\ue41d':'4',
    r'\ue439':'0',
    r'\ue45d':'9',
    r'\ue463':'2',
    r'\ue49d':'3',
    r'\ue4e3':'8',
    r'\ue534':'6',
    r'\ue54c':'1',
    r'\ue411':'5',
    r'\ue40a':'万'
}

def replace_(col):
    for key, value in wordMap.items():
        col = re.sub(key, value, col)
    return col

print(replace_('\ue463\ue54c.\ue45d\ue4e3\ue40a'))

print(replace_('\ue463.\ue534\ue411\ue40a'))

21.98万
2.65万


In [3]:
import os
# find all json files
json_files = []
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.json'):
            json_files.append(os.path.join(root, file))
json_files[:5]

['.\\json\\cars1.json',
 '.\\json\\cars10.json',
 '.\\json\\cars100.json',
 '.\\json\\cars101.json',
 '.\\json\\cars102.json']

In [4]:
# read json file from html
import json
import pandas as pd
data = []
for jsonPath in json_files:
    with open(jsonPath, 'r', encoding='utf-8') as f_:
        car_data = json.load(f_)
        # header names
        Headers = [
            "brand_name"
            , "car_name"
            , "car_year"
            , "official_price"
            , "sh_price"]

        data_searched = car_data['data']['search_sh_sku_info_list']

        # memory data

        # print(cars)

        for diclst in data_searched:

            # print(replace_(diclst['sh_price']))

            #还可以使用列表生成式
            # dit = [diclst[col]  if col not in 'official_price and sh_price' else replace_(diclst[col]) for col in Headers]
            dit = {col: diclst[col]for col in Headers}
            # print(dit)
            data.append(dit)

        cars = pd.DataFrame(columns=Headers, data=data)




In [5]:
cars

,brand_name,car_name,car_year,official_price,sh_price
0,比亚迪,1.5T 双冠旗舰Plus版,2015,.,.
1,比亚迪,秦EV300 尊享型,2017,.,.
2,吉利汽车,2.0L 自动尊贵型,2014,.,.
3,众泰,1.8T 双离合尊享型,2017,.,.
4,路虎,2.0T 5门耀真版,2013,.,.
...,...,...,...,...,...
9995,沃尔沃,B4 智逸豪华版,2022,.,.
9996,奥迪,40 TFSI 豪华动感型,2022,.,.
9997,哈弗,1.5T 手动两驱精英型 国VI,2019,.,.
9998,本田,240 TURBO CVT 尊享版,2020,.,.


In [6]:
# you can also use dataframe-function apply
cars['sh_price'] = cars['sh_price'].apply(replace_)
cars['official_price'] = cars['official_price'].apply(replace_)




In [7]:
cars_duplicated = cars.drop_duplicates()
cars_duplicated.to_csv('./resource/used_cars.csv', index=False, header=True)


In [11]:
cars_duplicated.sort_values('brand_name', ignore_index=True, inplace=True)
cars_duplicated


C:\Users\sunsh\AppData\Local\Temp\ipykernel_9684\4024794391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_duplicated.sort_values('brand_name', ignore_index=True, inplace=True)


,brand_name,car_name,car_year,official_price,sh_price
0,AITO,еўһзЁӢеӣӣй©ұжҷәй©ҫзүҲ,2023,о‘Јо‘қ.о‘қо“ЈоҗҠ,о•Ңо‘қ.о’қо“ЈоҗҠ
1,AITO,Plus五座后驱版,2024,24.98万,19.98万
2,AITO,Max五座四驱智驾版,2024,30.98万,23.58万
3,AITO,Ultra增程版,2024,52.98万,52.98万
4,AITO,Max六座四驱智驾版,2024,32.98万,26.80万
...,...,...,...,...,...
9208,魏牌,1.5T 智先锋,2020,13.78万,6.58万
9209,魏牌,2.0T 超豪型,2017,17.78万,6.88万
9210,魏牌,1.5T 元气橙版,2021,14.68万,6.68万
9211,魏牌,四驱旗舰版,2023,37.58万,29.60万
